# Get Dataset

In [ ]:
# Get roboflow data for pickleball court keypoints detection training
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KEYHERE")
project = rf.workspace("pickleball-ball-detection").project("pickleball-court-keypoints-syncz")
version = project.version(6)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Pickleball-Court-Keypoints-6 in coco:: 100%|██████████| 49/49 [00:00<00:00, 2191.29it/s]


# Start Code

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [24]:
class keyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # Adjust keypoints
        kps[::2] *= (224 / w) # Adjust x coordinates
        kps[1::2] *= (224 / h) # Adjust y coordinates

        return img, kps
    

In [25]:
# Adjust file paths
# Using RoboFlow JSON COCO format -> adapt to your tutorial's JSON ("id", "kps")
from pathlib import Path
import json
from PIL import Image

root = Path(dataset.location)
inner = root / "Pickleball-1" if (root / "Pickleball-1").exists() else root  # handle earlier nesting

rf_train_images = inner / "train"
rf_val_images   = inner / ("valid" if (inner / "valid").exists() else "val")
rf_train_ann    = rf_train_images / "_annotations.coco.json"
rf_val_ann      = rf_val_images   / "_annotations.coco.json"

print("Roboflow train:", rf_train_images, rf_train_ann)
print("Roboflow val  :", rf_val_images,   rf_val_ann)

out_root         = Path("data")
out_root.mkdir(exist_ok=True)
out_train_images = out_root / "images"
out_val_images   = out_root / "images_val"
out_train_json   = out_root / "data_train.json"
out_val_json     = out_root / "data_val.json"
out_train_images.mkdir(parents=True, exist_ok=True)
out_val_images.mkdir(parents=True, exist_ok=True)

def export_split(src_img_dir: Path, coco_json: Path, dst_img_dir: Path, dst_json: Path) -> int:
    data = json.load(open(coco_json))
    id2file = {im["id"]: im["file_name"] for im in data["images"]}

    # pick one annotation per image (most keypoints wins)
    by_img = {}
    for a in data["annotations"]:
        if "keypoints" not in a or not a["keypoints"]:
            continue
        img_id = a["image_id"]
        p = by_img.get(img_id)
        if p is None or a.get("num_keypoints", 0) > p.get("num_keypoints", 0):
            by_img[img_id] = a

    items = []
    for img_id, fname in id2file.items():
        if img_id not in by_img:
            continue
        src = src_img_dir / fname
        alt = src_img_dir / "images" / fname
        if not src.exists() and alt.exists():
            src = alt
        if not src.exists():
            print(f"⚠️ missing image: {src}")
            continue

        stem = Path(fname).stem
        dst  = dst_img_dir / f"{stem}.png"  # your loader uses f"{id}.png"
        Image.open(src).convert("RGB").save(dst)

        raw = by_img[img_id]["keypoints"]         # [x1,y1,v1, x2,y2,v2, ...]
        # Build the exact field your dataset reads:
        kps_xy = [v for i, v in enumerate(raw) if (i % 3) != 2]   # drop visibility -> [x1,y1,x2,y2,...]

        # Write BOTH names for safety: your code uses "kps"
        items.append({"id": stem, "kps": kps_xy, "keypoints": raw})

    json.dump(items, open(dst_json, "w"))
    print(f"✅ Wrote {len(items)} -> {dst_img_dir} and {dst_json}")
    return len(items)

export_split(rf_train_images, rf_train_ann, out_train_images, out_train_json)
export_split(rf_val_images,   rf_val_ann,   out_val_images,   out_val_json)

# Keep your variable names untouched
train_images = out_train_images
val_images   = out_val_images
train_ann    = out_train_json
val_ann      = out_val_json

print("train_images:", train_images)
print("train_ann   :", train_ann)
print("val_images  :", val_images)
print("val_ann     :", val_ann)



Roboflow train: /Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/train /Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/train/_annotations.coco.json
Roboflow val  : /Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/valid /Users/darensivam/Desktop/Projects/pickleball/training/Pickleball-Court-Keypoints-6/valid/_annotations.coco.json
✅ Wrote 29 -> data/images and data/data_train.json
✅ Wrote 8 -> data/images_val and data/data_val.json
train_images: data/images
train_ann   : data/data_train.json
val_images  : data/images_val
val_ann     : data/data_val.json


In [26]:
# Obtain datasets
train_dataset = keyPointsDataset(str(train_images), str(train_ann))
val_dataset   = keyPointsDataset(str(val_images), str(val_ann))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [27]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 12*2) # Replaces the last layer for 12 keypoints (x, y) pairs

In [28]:
# Move model to device
model = model.to(device)

# Train Model

In [39]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [45]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs, kps = imgs.to(device), kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}")

Epoch [1/20], Step [1/4], Loss: 7.342075347900391
Epoch [2/20], Step [1/4], Loss: 6.188163757324219
Epoch [3/20], Step [1/4], Loss: 14.398063659667969
Epoch [4/20], Step [1/4], Loss: 14.615649223327637
Epoch [5/20], Step [1/4], Loss: 4.360602855682373
Epoch [6/20], Step [1/4], Loss: 6.405500411987305
Epoch [7/20], Step [1/4], Loss: 4.837834358215332
Epoch [8/20], Step [1/4], Loss: 6.626933574676514
Epoch [9/20], Step [1/4], Loss: 5.758828639984131
Epoch [10/20], Step [1/4], Loss: 16.11936378479004
Epoch [11/20], Step [1/4], Loss: 5.513871669769287
Epoch [12/20], Step [1/4], Loss: 7.7306036949157715
Epoch [13/20], Step [1/4], Loss: 5.3212571144104
Epoch [14/20], Step [1/4], Loss: 3.9121315479278564
Epoch [15/20], Step [1/4], Loss: 13.30227279663086
Epoch [16/20], Step [1/4], Loss: 3.5669429302215576
Epoch [17/20], Step [1/4], Loss: 4.656815052032471
Epoch [18/20], Step [1/4], Loss: 3.633469581604004
Epoch [19/20], Step [1/4], Loss: 15.32172679901123
Epoch [20/20], Step [1/4], Loss: 13.0

In [46]:
# Save the model
torch.save(model.state_dict(), "keypoint_model.pth")